In [40]:
""" USEFUL WEBSITES
https://www.kaggle.com/code/ravi20076/playgrounds5e01-public-baseline-v1#KEY-NOTE
https://www.kaggle.com/competitions/playground-series-s5e1/discussion/554349
https://www.kaggle.com/code/curtiscoding1/linear-reg-model-analysis-walk-forward-val#Model-Analysis-&-Validation
https://www.kaggle.com/code/jiaoyouzhang/fss-lightgbm-holidays-fitter
https://www.kaggle.com/code/sunilkumarmuduli/complete-tutorial-model-time-series-eda

Current plan, try elastinet using optuna to test which ratio works best

submission did rather poorly, I think it may be overfitted cause the cv_score performed siginificantly better
I will impute the values first, then I may attempt to check for multicolliearity unless someone else 
decides to do that sooner.
https://www.kaggle.com/code/sidcodegg/linearmodel-keep-it-simple-silly
use this model and do forward backward selection for ridge cause this model did quite poor
"""
"""
Do competition below immediately after this one finishes as it is similar
https://www.kaggle.com/competitions/playground-series-s5e1/discussion/554772
"""


In [39]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import holidays
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("train.csv")
df["date"] = pd.to_datetime(df["date"])

In [40]:
df.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [41]:
print(df.dtypes)
print(df.shape)

id                   int64
date        datetime64[ns]
country             object
store               object
product             object
num_sold           float64
dtype: object
(230130, 6)


In [4]:
print(df.isnull().sum()/len(df) * 100)
print(df.duplicated().sum())

id          0.000000
date        0.000000
country     0.000000
store       0.000000
product     0.000000
num_sold    3.854778
dtype: float64
0


In [7]:
for i in df.columns:
    if (i == "id") or (i == "num_sold"):
        continue
    print(i + " Unique Values: " + str(df[i].nunique()))

date Unique Values: 2557
country Unique Values: 6
store Unique Values: 3
product Unique Values: 5


In [12]:
for i in df.columns:
    if i == "id" or i == "num_sold":
        continue
    print(df[i].value_counts())
    print("")

date
2010-01-01    90
2014-09-05    90
2014-08-29    90
2014-08-30    90
2014-08-31    90
              ..
2012-05-03    90
2012-05-04    90
2012-05-05    90
2012-05-06    90
2016-12-31    90
Name: count, Length: 2557, dtype: int64

country
Canada       38355
Finland      38355
Italy        38355
Kenya        38355
Norway       38355
Singapore    38355
Name: count, dtype: int64

store
Discount Stickers       76710
Stickers for Less       76710
Premium Sticker Mart    76710
Name: count, dtype: int64

product
Holographic Goose     46026
Kaggle                46026
Kaggle Tiers          46026
Kerneler              46026
Kerneler Dark Mode    46026
Name: count, dtype: int64



In [26]:
for i in dfTest.columns:
    if i == "id" or i == "num_sold":
        continue
    print(dfTest[i].value_counts())
    print("")

date
2017-01-01    90
2018-12-30    90
2019-01-05    90
2019-01-04    90
2019-01-03    90
              ..
2018-01-07    90
2018-01-08    90
2018-01-09    90
2018-01-10    90
2019-12-31    90
Name: count, Length: 1095, dtype: int64

country
Canada       16425
Finland      16425
Italy        16425
Kenya        16425
Norway       16425
Singapore    16425
Name: count, dtype: int64

store
Discount Stickers       32850
Stickers for Less       32850
Premium Sticker Mart    32850
Name: count, dtype: int64

product
Holographic Goose     19710
Kaggle                19710
Kaggle Tiers          19710
Kerneler              19710
Kerneler Dark Mode    19710
Name: count, dtype: int64

year
2017    32850
2018    32850
2019    32850
Name: count, dtype: int64

month
1     8370
3     8370
5     8370
7     8370
8     8370
10    8370
12    8370
4     8100
6     8100
9     8100
11    8100
2     7560
Name: count, dtype: int64

day
1     3240
2     3240
28    3240
27    3240
26    3240
25    3240
24    3240


In [42]:
""" Credited to this https://www.kaggle.com/code/cabaxiom/s5e1-eda-and-linear-regression-baseline?scriptVersionId=216127469
I thought about creating this myself but there isn't really any poitn to that if this is shared, from future onwards
using ratios of relavant data as a means of imputing is viable and the best-performing datasets do this and either use lienar regression or fine-tune using optuna"""
gdp_per_capita_df = pd.read_csv("gdp.csv")

years =  ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]
gdp_per_capita_filtered_df = gdp_per_capita_df.loc[gdp_per_capita_df["Country Name"].isin(df["country"].unique()), ["Country Name"] + years].set_index("Country Name")
gdp_to_merge = gdp_per_capita_filtered_df
gdp_per_capita_filtered_df["2010_ratio"] = gdp_per_capita_filtered_df["2010"] / gdp_per_capita_filtered_df.sum()["2010"]
for year in years:
    gdp_per_capita_filtered_df[f"{year}_ratio"] = gdp_per_capita_filtered_df[year] / gdp_per_capita_filtered_df.sum()[year]
gdp_per_capita_filtered_ratios_df = gdp_per_capita_filtered_df[[i+"_ratio" for i in years]]
gdp_per_capita_filtered_ratios_df.columns = [int(i) for i in years]
gdp_per_capita_filtered_ratios_df = gdp_per_capita_filtered_ratios_df.unstack().reset_index().rename(columns = {"level_0": "year", 0: "ratio", "Country Name": "country"})
gdp_per_capita_filtered_ratios_df['year'] = pd.to_datetime(gdp_per_capita_filtered_ratios_df['year'], format='%Y')
gdp_per_capita_filtered_df

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2011_ratio,2012_ratio,2013_ratio,2014_ratio,2015_ratio,2016_ratio,2017_ratio,2018_ratio,2019_ratio,2020_ratio
Country Name,,,,,,,,,,,,,,,,,,,,,
Canada,47560.666600,52223.858840,52670.344730,52638.118720,50960.843120,43594.194100,42314.061580,45129.628120,46539.176160,46352.869340,...,0.175045,0.178867,0.175479,0.173632,0.175439,0.171860,0.171772,0.164793,0.169793,0.169942
Finland,46506.291900,51060.324260,47551.674080,49691.014520,50073.776080,42560.345680,43451.256240,46085.017470,49654.249700,48358.180780,...,0.171145,0.161484,0.165654,0.170609,0.171278,0.176478,0.175408,0.175823,0.177138,0.190594
Italy,36184.711870,38851.388130,35235.798910,35747.707950,35750.719750,30387.129320,31126.324690,32554.146680,34746.344140,33812.788720,...,0.130223,0.119660,0.119171,0.121808,0.122289,0.126420,0.123907,0.123035,0.123858,0.125263
Kenya,1091.518731,1096.144998,1285.005362,1370.878852,1482.815663,1489.119598,1554.126103,1667.484472,1836.452755,1960.408089,...,0.003674,0.004364,0.004570,0.005052,0.005993,0.006312,0.006347,0.006503,0.007181,0.007524
Norway,88163.208590,101221.813500,102175.919300,103553.840100,97666.695180,74809.965800,70867.361000,76131.838400,82792.842710,76430.588950,...,0.339278,0.346987,0.345215,0.332766,0.301063,0.287829,0.289772,0.293166,0.279969,0.266753
Singapore,47236.683080,53891.457030,55547.555310,56967.425790,57564.802310,55645.606860,56899.918180,61162.097390,66840.637340,66081.719920,...,0.180635,0.188638,0.189911,0.196132,0.223938,0.231100,0.232794,0.236680,0.242061,0.239924


In [43]:
""" credit https://www.kaggle.com/code/cabaxiom/s5e1-eda-and-linear-regression-baseline?scriptVersionId=216127469 """
gdp_per_capita_filtered_ratios_df["year"] = gdp_per_capita_filtered_ratios_df["year"].dt.year
df_imputed = df.copy()
print(f"Missing values remaining: {df_imputed['num_sold'].isna().sum()}")

df_imputed["year"] = df_imputed["date"].dt.year
for year in df_imputed["year"].unique():
    # Impute Time Series 1 (Canada, Discount Stickers, Holographic Goose)
    target_ratio = gdp_per_capita_filtered_ratios_df.loc[(gdp_per_capita_filtered_ratios_df["year"] == year) & (gdp_per_capita_filtered_ratios_df["country"] == "Norway"), "ratio"].values[0] # Using Norway as should have the best precision
    current_raito = gdp_per_capita_filtered_ratios_df.loc[(gdp_per_capita_filtered_ratios_df["year"] == year) & (gdp_per_capita_filtered_ratios_df["country"] == "Canada"), "ratio"].values[0]
    ratio_can = current_raito / target_ratio
    df_imputed.loc[(df_imputed["country"] == "Canada") & (df_imputed["store"] == "Discount Stickers") & 
    (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year), "num_sold"] = (df_imputed.loc[(df_imputed["country"] == "Norway") & (df_imputed["store"] == "Discount Stickers") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year), "num_sold"] * ratio_can).values
    
    # Impute Time Series 2 (Only Missing Values)
    current_ts =  df_imputed.loc[(df_imputed["country"] == "Canada") & (df_imputed["store"] == "Premium Sticker Mart") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year)]
    missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
    df_imputed.loc[(df_imputed["country"] == "Canada") & (df_imputed["store"] == "Premium Sticker Mart") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] = (df_imputed.loc[(df_imputed["country"] == "Norway") & (df_imputed["store"] == "Premium Sticker Mart") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] * ratio_can).values

    # Impute Time Series 3 (Only Missing Values)
    current_ts =  df_imputed.loc[(df_imputed["country"] == "Canada") & (df_imputed["store"] == "Stickers for Less") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year)]
    missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
    df_imputed.loc[(df_imputed["country"] == "Canada") & (df_imputed["store"] == "Stickers for Less") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] = (df_imputed.loc[(df_imputed["country"] == "Norway") & (df_imputed["store"] == "Stickers for Less") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] * ratio_can).values
    
    # Impute Time Series 4 (Kenya, Discount Stickers, Holographic Goose)
    current_raito = gdp_per_capita_filtered_ratios_df.loc[(gdp_per_capita_filtered_ratios_df["year"] == year) & (gdp_per_capita_filtered_ratios_df["country"] == "Kenya"), "ratio"].values[0]
    ratio_ken = current_raito / target_ratio
    df_imputed.loc[(df_imputed["country"] == "Kenya") & (df_imputed["store"] == "Discount Stickers") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year), "num_sold"] = (df_imputed.loc[(df_imputed["country"] == "Norway") & (df_imputed["store"] == "Discount Stickers") & (df_imputed["product"] == "Holographic Goose")& (df_imputed["year"] == year), "num_sold"] * ratio_ken).values

    # Impute Time Series 5 (Only Missing Values)
    current_ts = df_imputed.loc[(df_imputed["country"] == "Kenya") & (df_imputed["store"] == "Premium Sticker Mart") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year)]
    missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
    df_imputed.loc[(df_imputed["country"] == "Kenya") & (df_imputed["store"] == "Premium Sticker Mart") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] = (df_imputed.loc[(df_imputed["country"] == "Norway") & (df_imputed["store"] == "Premium Sticker Mart") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] * ratio_ken).values

    # Impute Time Series 6 (Only Missing Values)
    current_ts = df_imputed.loc[(df_imputed["country"] == "Kenya") & (df_imputed["store"] == "Stickers for Less") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year)]
    missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
    df_imputed.loc[(df_imputed["country"] == "Kenya") & (df_imputed["store"] == "Stickers for Less") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] = (df_imputed.loc[(df_imputed["country"] == "Norway") & (df_imputed["store"] == "Stickers for Less") & (df_imputed["product"] == "Holographic Goose") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] * ratio_ken).values

    # Impute Time Series 7 (Only Missing Values)
    current_ts = df_imputed.loc[(df_imputed["country"] == "Kenya") & (df_imputed["store"] == "Discount Stickers") & (df_imputed["product"] == "Kerneler") & (df_imputed["year"] == year)]
    missing_ts_dates = current_ts.loc[current_ts["num_sold"].isna(), "date"]
    df_imputed.loc[(df_imputed["country"] == "Kenya") & (df_imputed["store"] == "Discount Stickers") & (df_imputed["product"] == "Kerneler") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] = (df_imputed.loc[(df_imputed["country"] == "Norway") & (df_imputed["store"] == "Discount Stickers") & (df_imputed["product"] == "Kerneler") & (df_imputed["year"] == year) & (df_imputed["date"].isin(missing_ts_dates)), "num_sold"] * ratio_ken).values
    
print(f"Missing values remaining: {df_imputed['num_sold'].isna().sum()}")

Missing values remaining: 8871
Missing values remaining: 2


In [44]:
""" Credit to https://www.kaggle.com/code/cabaxiom/s5e1-eda-and-linear-regression-baseline?scriptVersionId=216127469 """
missing_rows = df_imputed.loc[df_imputed["num_sold"].isna()]

df_imputed.loc[df_imputed["id"] == 23719, "num_sold"] = 4
df_imputed.loc[df_imputed["id"] == 207003, "num_sold"] = 195

print(f"Missing values remaining: {df_imputed['num_sold'].isna().sum()}")
df = df_imputed

Missing values remaining: 0


In [45]:
gdp_to_merge = gdp_to_merge[["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]].reset_index().melt(id_vars=['Country Name'], var_name='year', value_name="gdp")
gdp_to_merge = gdp_to_merge.rename(columns={"Country Name": "country"})

gdp_to_merge.head()

,country,year,gdp
0,Canada,2010,47560.666600
1,Finland,2010,46506.291900
2,Italy,2010,36184.711870
3,Kenya,2010,1091.518731
4,Norway,2010,88163.208590


In [46]:
def dmy_splitter(df):
    df["date"] = pd.to_datetime(df["date"])
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    df["day"] = df["date"].dt.day

    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekofyear'] = df['date'].dt.isocalendar().week



    df['month_product'] = df['month'].astype(str) + "_" + df['product']
    df['month_product'] = df['month_product'].astype("category")
    df['dayofweek'] = df['dayofweek'].apply(
        lambda x: 0 if x in [0,1,2,3] else
        1 if x == 4 else
        2 if x == 5 else
        3).astype("category")
    df["country"] = df["country"].astype("category")
    df["store"] = df["store"].astype("category")
    df["is_holiday"] = df["is_holiday"].astype("category")
    return df

holiday_calendars = {
    "Canada": holidays.Canada(years=2015),
    "Finland": holidays.Finland(years=2015),
    "Italy": holidays.Italy(years=2015),
    "Norway": holidays.Norway(years=2015),
    "Singapore": holidays.Singapore(years=2015),
    "Kenya": holidays.Kenya(years=2015)
}

def is_holiday(row):
    country_holidays = holiday_calendars.get(row["country"])
    if country_holidays:
        return row["date"] in country_holidays
    return False


df["is_holiday"] = df.apply(is_holiday, axis = 1)

df = dmy_splitter(df)
df.head()

,id,date,country,store,product,num_sold,year,is_holiday,month,day,dayofweek,weekofyear,month_product
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,140.799481,2010,True,1,1,1,53,1_Holographic Goose
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.000000,2010,True,1,1,1,53,1_Kaggle
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.000000,2010,True,1,1,1,53,1_Kaggle Tiers
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.000000,2010,True,1,1,1,53,1_Kerneler
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.000000,2010,True,1,1,1,53,1_Kerneler Dark Mode


In [47]:
gdp_to_merge["country"] = gdp_to_merge["country"].astype("category")
gdp_to_merge["year"] = gdp_to_merge["year"].astype("int32")
df = pd.merge(df, gdp_to_merge, on=['country', 'year'], how='inner')
df.head()

,id,date,country,store,product,num_sold,year,is_holiday,month,day,dayofweek,weekofyear,month_product,gdp
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,140.799481,2010,True,1,1,1,53,1_Holographic Goose,47560.6666
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.000000,2010,True,1,1,1,53,1_Kaggle,47560.6666
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.000000,2010,True,1,1,1,53,1_Kaggle Tiers,47560.6666
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.000000,2010,True,1,1,1,53,1_Kerneler,47560.6666
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.000000,2010,True,1,1,1,53,1_Kerneler Dark Mode,47560.6666


In [14]:
for i in df[df.isna().any(axis=1)].columns:
    if (i == "id") or (i == "num_sold"):
        continue
    print(i + " Unique Values: " + str(df[i].nunique()))

date Unique Values: 2557
country Unique Values: 6
store Unique Values: 3
product Unique Values: 5


In [48]:
X = df.drop(columns = ["id", "num_sold", "date", "product"])
y = np.log1p(df["num_sold"])

In [51]:
param_grid = {
    'alpha': np.linspace(0.001, 1, 20),
    'l1_ratio': np.linspace(0.001, 1, 20)
}

categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(exclude=['object', 'category']).columns
X[numerical_cols] = StandardScaler().fit_transform(X[numerical_cols])
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
tscv = TimeSeriesSplit(n_splits=5)

"""for fold, (train_idx, val_idx) in enumerate(tscv.split(X, y), 1):
    print(f"Fold {fold}:")
    print(f"  Training years: {X['year'].iloc[train_idx].unique()}")
    print(f"  Validation years: {X['year'].iloc[val_idx].unique()}")
    print("-" * 40)
"""
grid_search = GridSearchCV(ElasticNet(random_state=42), param_grid, cv = TimeSeriesSplit(n_splits=5), scoring='neg_mean_absolute_percentage_error')
grid_search.fit(X,y)
grid_search.best_params_

C:\ProgramData\anaconda3\envs\dataset_pytorch_based\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.827e+02, tolerance: 1.058e+01
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\envs\dataset_pytorch_based\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.399e+02, tolerance: 2.149e+01
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\envs\dataset_pytorch_based\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

{'alpha': 0.001, 'l1_ratio': 0.001}

In [52]:
grid_search.best_score_

-0.020360057350073936

In [84]:
dfTest = pd.read_csv("test.csv")
dfTest.head()


,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode


In [85]:
dfTest["is_holiday"] = dfTest.apply(is_holiday, axis = 1)
dfTest = dmy_splitter(dfTest)
dfTest = pd.merge(dfTest, gdp_to_merge, on=['country', 'year'], how='inner')
dfTest[dfTest.select_dtypes(include=["object"]).columns] = dfTest.select_dtypes(include=["object"]).astype("category")
dfTest.head()

,id,date,country,store,product,is_holiday,month,year,day,dayofweek,weekofyear,month_product,gdp
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose,True,1,2017,1,3,52,1_Holographic Goose,45129.62812
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle,True,1,2017,1,3,52,1_Kaggle,45129.62812
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers,True,1,2017,1,3,52,1_Kaggle Tiers,45129.62812
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler,True,1,2017,1,3,52,1_Kerneler,45129.62812
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode,True,1,2017,1,3,52,1_Kerneler Dark Mode,45129.62812


year                            int32
day                             int32
day_sin                       float64
day_cos                       float64
weekofyear                     UInt32
week_of_year_sin              Float64
week_of_year_cos              Float64
month_sin                     float64
month_cos                     float64
gdp                           float64
country_Finland                  bool
country_Italy                    bool
country_Kenya                    bool
country_Norway                   bool
country_Singapore                bool
store_Premium Sticker Mart       bool
store_Stickers for Less          bool
product_Kaggle                   bool
product_Kaggle Tiers             bool
product_Kerneler                 bool
product_Kerneler Dark Mode       bool
is_holiday_True                  bool
month_2                          bool
month_3                          bool
month_4                          bool
month_5                          bool
month_6     

In [87]:

dfTestEncoded = dfTest.drop(columns = ["id", "date", "product"])
categorical_cols = dfTestEncoded.select_dtypes(include=['object', 'category']).columns
dfTestEncoded = pd.get_dummies(dfTestEncoded, columns=categorical_cols, drop_first=True)
numerical_cols = dfTestEncoded.select_dtypes(exclude=['object', 'category']).columns
dfTestEncoded[numerical_cols] = StandardScaler().fit_transform(dfTestEncoded[numerical_cols])

In [97]:
y_test_pred = np.expm1(pd.DataFrame({'Premium Amount': grid_search.best_estimator_.predict(dfTestEncoded[grid_search.best_estimator_.feature_names_in_])}))

In [98]:
y_test_pred["id"] = dfTest['id']
y_test_pred = y_test_pred[['id', 'Premium Amount']]
y_test_pred.to_csv("submission5.csv", index = False)
